<a href="https://colab.research.google.com/github/playmobil/quick-learn-shell/blob/main/%E7%88%AC%E8%99%AB%E6%B5%8B%E8%AF%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from pathlib import Path
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video
from diffusers.models.attention_processor import AttnProcessor
from safetensors import safe_open
import gc
import requests

lcm_scheduler_url = "https://huggingface.co/spaces/wangfuyun/AnimateLCM-SVD/raw/main/lcm_scheduler.py"

r = requests.get(lcm_scheduler_url)

with open("lcm_scheduler.py", "w") as f:
    f.write(r.text)

from lcm_scheduler import AnimateLCMSVDStochasticIterativeScheduler
from huggingface_hub import hf_hub_download

MODEL_DIR = Path("model")

IMAGE_ENCODER_PATH = MODEL_DIR / "image_encoder.xml"
VAE_ENCODER_PATH = MODEL_DIR / "vae_encoder.xml"
VAE_DECODER_PATH = MODEL_DIR / "vae_decoder.xml"
UNET_PATH = MODEL_DIR / "unet.xml"


load_pt_pipeline = not (VAE_ENCODER_PATH.exists() and VAE_DECODER_PATH.exists() and UNET_PATH.exists() and IMAGE_ENCODER_PATH.exists())

unet, vae, image_encoder = None, None, None
if load_pt_pipeline:
    noise_scheduler = AnimateLCMSVDStochasticIterativeScheduler(
        num_train_timesteps=40,
        sigma_min=0.002,
        sigma_max=700.0,
        sigma_data=1.0,
        s_noise=1.0,
        rho=7,
        clip_denoised=False,
    )
    pipe = StableVideoDiffusionPipeline.from_pretrained(
        "stabilityai/stable-video-diffusion-img2vid-xt",
        variant="fp16",
        scheduler=noise_scheduler,
    )
    pipe.unet.set_attn_processor(AttnProcessor())
    hf_hub_download(
        repo_id="wangfuyun/AnimateLCM-SVD-xt",
        filename="AnimateLCM-SVD-xt.safetensors",
        local_dir="./checkpoints",
    )
    state_dict = {}
    LCM_LORA_PATH = Path(
        "checkpoints/AnimateLCM-SVD-xt.safetensors",
    )
    with safe_open(LCM_LORA_PATH, framework="pt", device="cpu") as f:
        for key in f.keys():
            state_dict[key] = f.get_tensor(key)
    missing, unexpected = pipe.unet.load_state_dict(state_dict, strict=True)

    pipe.scheduler.save_pretrained(MODEL_DIR / "scheduler")
    pipe.feature_extractor.save_pretrained(MODEL_DIR / "feature_extractor")
    unet = pipe.unet
    unet.eval()
    vae = pipe.vae
    vae.eval()
    image_encoder = pipe.image_encoder
    image_encoder.eval()
    del pipe
    gc.collect()

# Load the conditioning image
image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/svd/rocket.png?download=true")
image = image.resize((512, 256))

RuntimeError: Failed to import diffusers.pipelines.stable_video_diffusion.pipeline_stable_video_diffusion because of the following error (look up to see its traceback):
Failed to import transformers.models.clip.image_processing_clip because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time

def download_unsplash_thumbnails(url, save_dir='unsplash_thumbnails'):
    # 创建保存目录
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 设置header模拟浏览器
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        # 获取页面内容
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # 找到所有图片元素
        img_elements = soup.find_all('img', {'class': 'YVj9w'})

        # 下载每个缩略图
        for i, img in enumerate(img_elements):
            if 'src' in img.attrs:
                img_url = img['src']

                # 获取图片内容
                img_response = requests.get(img_url, headers=headers)
                img_response.raise_for_status()

                # 保存图片
                file_name = f'museum_thumbnail_{i}.jpg'
                file_path = os.path.join(save_dir, file_name)

                with open(file_path, 'wb') as f:
                    f.write(img_response.content)

                print(f'Downloaded: {file_name}')

                # 添加延时，避免请求过快
                time.sleep(1)

        print('Download completed!')

    except Exception as e:
        print(f'An error occurred: {str(e)}')

# 使用示例
url = 'https://unsplash.com/s/photos/museum'
download_unsplash_thumbnails(url)

An error occurred: 403 Client Error: Forbidden for url: https://unsplash.com/s/photos/museum


In [3]:
%pip install selenium requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os
import time
from urllib.parse import urlparse

def get_file_size(response):
    # 获取文件大小（以字节为单位）
    return len(response.content)

def download_unsplash_thumbnails(url, save_dir='unsplash_thumbnails', min_size_kb=50):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 配置Chrome选项
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)

        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, "img"))
        )

        # 滚动页面加载更多图片
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_count = 0
        max_scrolls = 5  # 限制滚动次数，避免加载太多

        while scroll_count < max_scrolls:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            scroll_count += 1

        img_elements = driver.find_elements(By.TAG_NAME, "img")

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Referer': url
        }

        downloaded_urls = set()
        successful_downloads = 0

        for i, img in enumerate(img_elements):
            try:
                img_url = img.get_attribute('src')

                if not img_url or img_url in downloaded_urls or not img_url.startswith('http'):
                    continue

                if 'unsplash.com' not in img_url:
                    continue

                # 先检查文件大小
                img_response = requests.get(img_url, headers=headers)
                img_response.raise_for_status()

                file_size_kb = get_file_size(img_response) / 1024  # 转换为KB

                # 只下载大于指定大小的图片
                if file_size_kb < min_size_kb:
                    print(f'Skipping image {i}: Size {file_size_kb:.2f}KB < {min_size_kb}KB')
                    continue

                file_ext = os.path.splitext(urlparse(img_url).path)[1]
                if not file_ext:
                    file_ext = '.jpg'

                file_name = f'museum_thumbnail_{successful_downloads}{file_ext}'
                file_path = os.path.join(save_dir, file_name)

                with open(file_path, 'wb') as f:
                    f.write(img_response.content)

                downloaded_urls.add(img_url)
                successful_downloads += 1
                print(f'Downloaded: {file_name} (Size: {file_size_kb:.2f}KB)')

                time.sleep(1)

            except Exception as e:
                print(f'Error downloading image {i}: {str(e)}')
                continue

        print(f'Download completed! Total images downloaded: {successful_downloads}')

    except Exception as e:
        print(f'An error occurred: {str(e)}')

    finally:
        driver.quit()

# 使用示例
url = 'https://unsplash.com/s/photos/museum'
download_unsplash_thumbnails(url, min_size_kb=50)  # 设置最小文件大小为50KB

Downloaded: museum_thumbnail_0.jpg (Size: 832.81KB)
Skipping image 7: Size 3.79KB < 50KB
Downloaded: museum_thumbnail_1.jpg (Size: 480.09KB)
Skipping image 10: Size 7.29KB < 50KB
Downloaded: museum_thumbnail_2.jpg (Size: 321.94KB)
Skipping image 13: Size 14.17KB < 50KB
Downloaded: museum_thumbnail_3.jpg (Size: 863.98KB)
Skipping image 16: Size 7.36KB < 50KB
Downloaded: museum_thumbnail_4.jpg (Size: 1731.60KB)
Skipping image 19: Size 6.74KB < 50KB
Downloaded: museum_thumbnail_5.jpg (Size: 637.85KB)
Skipping image 22: Size 5.26KB < 50KB
Downloaded: museum_thumbnail_6.jpg (Size: 696.12KB)
Skipping image 25: Size 12.40KB < 50KB
Downloaded: museum_thumbnail_7.jpg (Size: 529.38KB)
Skipping image 28: Size 5.26KB < 50KB
Downloaded: museum_thumbnail_8.jpg (Size: 1067.34KB)
Skipping image 31: Size 7.06KB < 50KB
Downloaded: museum_thumbnail_9.jpg (Size: 62.45KB)
Skipping image 33: Size 6.25KB < 50KB
Downloaded: museum_thumbnail_10.jpg (Size: 421.57KB)
Skipping image 40: Size 2.77KB < 50KB
Downloa

# 新段落